# Statistical Testing 

In this notebook, I want to discuss the following question - When can I say that two variables are correlated? And how can I measure this correlation? This question is fundamental to any data analysis task. 

How can one tackle this question - One can look at the data for the two variables, plot them in some way and look at the correlation. This is actually quite important but this gives us only a qualitative idea. How do I get a quantitative answer to this question? 

The answer depends on the variables that I wish to consider - there are many forms of similarity in mathematics:
1. Inner Product - to compare the angle between two vectors
2. KL divergence - to compare two probability distributions
and many more.

Why not just use one of these measures? Well, first of all, the things that we want to compare are typically random variables. So, how do you define inner product of random variables? Well, you could consider the expectation of the inner product of these random variables. It is clear that we would have to normalize the random variables if we want this expectation to represent anything meaningful. In fact, this does turn out to be a meaningful measure of correlation - this is the pearon correlation coefficient. 

## Pearson correlation coefficient

The pearson correlation coefficient is given by:

$$\rho_{X,Y} = \frac{\mathbb{E}[(X-\mu_X) (Y-\mu_Y)]}{\sigma_X \sigma_Y}$$

This will always be between -1 and 1. 

Nice, so we have one such measure. But now there are more questions: 
1. Given some data, how do I actually estimate the correlation coefficient?
2. What are the advantages and limitations of pearson correlation coefficient? 

In python we can use the command `pearsonr(x,y)` to calculate this coefficient, where `x` and `y` are arrays containing the different instances of the two random variables $X$ and $Y$. 

### Example

Let $X$ be a uniform random variable over [0,1000). Let's consider 100 samples from it. Now, let's have $Y = 2X + e$ where $e$ is some error sampled from standard normal. In this case, the pearson coefficient should be high.

In [24]:
import numpy as np
from scipy import stats

size = 100
X = 1000*np.random.rand(size)
e = np.random.normal(0,1, size)
Y = 2*X+e 
stats.pearsonr(X,Y)

(0.9999980101527586, 1.9853438749369634e-266)

The output is in the format `(value, p-value)`. As mentioned in scipy documentation - "The p-value roughly indicates the probability of an uncorrelated system producing datasets that have a Pearson correlation at least as extreme as the one computed from these datasets." 

But we have to be careful, the calculation of this p-value assumes that $X$ and $Y$ are normally distributed. So, we shouldn't consider it in our analysis unless we know that $X$ and $Y$ are roughly normal. 

For completion, let's consider $X$ and $Y$ independent now.  

In [27]:
Y = 1000*np.random.rand(size)
stats.pearsonr(X,Y)

(0.13482319574714438, 0.18109325902693293)

## Limitations

There are following limitations to the pearson correlation coefficient calculations. Let's look at them one by one and illustrate them using python.

Pearson correlation coefficient captures only linear relationships.

We consider Y to be = X for half the instances and - X for half the instances. Clearly Y and X are completely correlated but this isn't captured by the pearson correlation coefficient.

In [45]:
Y = np.concatenate((X[0:int(size/2)] ,-X[int(size/2):size+1]))
stats.pearsonr(X,Y)

(-0.26237108697847694, 0.008361983859359266)

I have noticed that Pearson's correlation coefficient seems to be high even when the variables are correlated non-linearly but still monotonically. For example: $Y = X^2$ when $X > 0$.

In [46]:
Y = X**2
stats.pearsonr(X,Y)

(0.9727689945846026, 4.941045099223802e-64)

So, what's happening. This [paper](https://www.tandfonline.com/doi/full/10.1080/00031305.2021.2004922)  goes into what's happening. It seems that if there exist transformations $\phi, \psi$ such that $\mathbb{E}[\phi(Y)|\psi(X)] = \alpha\phi(X) + C$ then despite non-linear correlation the pearson correlation coefficient would work. For example, for $Y = X^2$, considering $\psi(X) = X^2, \phi(Y) = Y$, we get $\mathbb{E}[Y|\psi(X)] = \psi(X)$. I haven't gone very deep into this, so more reading is needed.

## Additional Remarks:
1. While applying statistical testing we should always be careful about the multiple testing issue. This occurs when I test many hypotheses at once. 